In [42]:
import pandas as pd 
from multiprocessing import Pool
from fuzzywuzzy import fuzz
import numpy as np

# read the data
acquirer_data = pd.read_excel("./data/acquirers.xlsx")
bank_data = pd.read_csv("./data/bank_names.csv")
output = pd.DataFrame(columns=["acquirer", "match1", "match2", "match3", "match4", "match5"])


In [29]:
bank_list = bank_data['bank_names'].tolist()

In [30]:
# fuzzy match
def fuzzy_match(index):
    match_list = []
    firm_name = acquirer_data.loc[index, 'Acquirer Name']
    for bank_name in bank_list:
        similarity = fuzz.ratio(firm_name, bank_name) + \
                    fuzz.partial_ratio(firm_name, bank_name) + \
                    fuzz.token_sort_ratio(firm_name, bank_name) + \
                    fuzz.token_set_ratio(firm_name, bank_name)
        match_list.append((similarity, bank_name))
    
    match_list.sort(key=lambda x: x[0], reverse=True)
    match = [element[1] for element in match_list[:5]]
    output.loc[index] = [firm_name] + match


In [45]:
# parallel processing
if __name__ == '__main__':
    pool = Pool(processes=4)
    pool.map(fuzzy_match, np.linspace(0, len(acquirer_data), 1, endpoint=False, dtype=int))
    pool.close()

In [38]:
fuzzy_match(3)

In [47]:
# parallelize the fuzzy match
if __name__ == '__main__':
    pool = Pool(4)
    pool.map(fuzzy_match, range(len(acquirer_data)))
    pool.start()
    pool.join()
